In [ ]:
exec_time_arr = []
subtasks = 0
quantum = 300
rejectQueue = []
for task in df.itertuples():
    taskID = task.TaskID
    burstTime = ((taskID%2)+1)*quantum      # only two options for burst time
    exec_time_arr.append(burstTime)
    subtasks += burstTime / quantum

time_frame = pd.DataFrame({
    'ExecutionTime': exec_time_arr,
    'ExecutionTime_Modified': exec_time_arr,
    'TurnaroundTime_Power': [-1]*len(df),
    'TurnaroundTime_Cost': [-1]*len(df),
    'TurnaroundTime': [-1]*len(df)
})
df = df.join(time_frame)
print("Subtasks:", subtasks)

In [ ]:
def greedy(task_queue, vms, optIdentifier, currTime):
    reject_queue = []
    #available_vms = []
    turnaroundtime_sum = 0
    turnaroundtime_count = 0
    for task_idx in task_queue:
        minOptDelta = 100000000000
        minPowerDelta = 0
        min_vm_idx = 0
        vm_space_found = False
        
        for vm_idx in range(len(vms)):
            if(df['NrmlTaskCores'][task_idx] <= vms[vm_idx][0]) and (df['NrmlTaskMem'][task_idx] <= vms[vm_idx][1]):
                vm_space_found = True
                if(optIdentifier == "power"):
                    optDelta = powerCalculator(vms,task_idx,vm_idx)
                    if(optDelta < minOptDelta):
                        minOptDelta = optDelta
                        min_vm_idx = vm_idx
                    # Make sure you first run the code that initializes the execution time / turnaround
                    # dataframe before running the greedy algorithm! Do this every time
                    if(df['ExecutionTime_Modified'][task_idx] == quantum):
                        # Turnaround time is the curr time that the task finishes minus the arrival time
                        finishTime = currTime + quantum
                        startTime = df['Time']
                        df['TurnaroundTime_Power'][task_idx] = finishTime - startTime
                        df['ExecutionTime_Modified'][task_idx] -= quantum
                    elif(df['ExecutionTime_Modified'][task_idx] > quantum):
                        df['ExecutionTime_Modified'][task_idx] -= quantum
                elif(optIdentifier == "cost"):
                    powerDelta = powerCalculator(vms,task_idx,vm_idx)
                    prcIdx = int((currTime - 90000)/3600)
                    optDelta = Price_t[prcIdx]*powerDelta
                    if(optDelta < minOptDelta):
                        minOptDelta = optDelta
                        min_vm_idx = vm_idx
                    # Make sure you first run the code that initializes the execution time / turnaround
                    # dataframe before running the greedy algorithm! Do this every time
                    if(df['ExecutionTime_Modified'][task_idx] == quantum):
                        # Turnaround time is the curr time that the task finishes minus the arrival time
                        finishTime = currTime + quantum
                        startTime = df['Time']
                        df['TurnaroundTime_Cost'][task_idx] = finishTime - startTime
                        df['ExecutionTime_Modified'][task_idx] -= quantum
                    elif(df['ExecutionTime_Modified'][task_idx] > quantum):
                        df['ExecutionTime_Modified'][task_idx] -= quantum
                elif(optIdentifier == "turnaroundtime"):
                    # After scheduling the shortest job task
                    finish_time = currTime + df['ExecutionTime']
                    start_time = df['Time']
                    df['TurnaroundTime'][task_idx] = finishTime - startTime
                        
        if(vm_space_found):
            vms[min_vm_idx][0] = vms[min_vm_idx][0] - df['NrmlTaskCores'][task_idx]
            vms[min_vm_idx][1] = vms[min_vm_idx][1] - df['NrmlTaskMem'][task_idx]
        else:
            reject_queue.append(task_idx)
    return reject_queue, vms
